# 基于条件随机场模型的中文分词算法

[![GitHub license](https://img.shields.io/github/license/Dragon1573/Revision-3A?style=for-the-badge)](https://github.com/Dragon1573/Revision-3A/blob/master/LICENSE)

## 准备语料库

准备一个已经分好词的、规模足够庞大的语料库，用于条件随机场模型的参数学习。语料库由大量类似下面的句子组成，词语之间采用空格符分隔。

> 尽管 印尼 中央 和 地方政府 已 派出 上千人 的 灭火队，但 由于 该 地区 长期 干旱 少 雨，所以 火势 至今 未 得到 有效 控制。

## 语料库特征初步学习

需要在语料库的每个词组中分析出每个字的状态，如“收益”需要改为“收\B 益\E”。对语料库中每个分好的词添加状态信息，标记后的语句如下：

> 尽\B 管\E 印\B 尼\E 中\B 央\E 和\S 地\S 方\M 政\M 府\E 已\S 派\B 出\E 上\B 千\M 人\E 的\S 灭\B 火\M 队\E ，\S 但\S 由\B 于\E 该\S 地\B 区\E 长\B 期\E 干\B 旱\E 少\S 雨\S ，\S 所\B 以\E 火\B 势\E 至\B 今\E 未\S 得\B 到\E 有\B 效\E 控\B 制\E 。\S

## 词语特征学习

词语特征学习是分词过程中非常重要的一步。

### 导入程序包，并定义全局变量

In [1]:
import json

LABEL = ('B', 'M', 'E', 'S')

### 输入需要分词的字符串

In [2]:
string = input('请输入目标语句：')

请输入目标语句： 希腊的经济结构较特殊


### 语料库预处理

语料库以标点符号区分每个句子，而在程序中，标点符号处不被认为是一个句子的开始/结束。所以我们需要对语料库执行预处理，将标点符号转换为起止标记。

In [3]:
# 读入语料库
with open('Training.txt', 'r', encoding='UTF-8') as database:
    document = database.readlines()
for line in range(len(document)):
    if ord(document[line][0]) < 0x4E00 or ord(document[line][0]) > 0x9FA5:
        document[line] = '^|S\n'
if document[0][0] != '^':
    document.insert(0, '^|S\n')
if document[-1][0] != '^':
    document.append('^|S\n')

### 统计各种字符在各种状态下出现的次数

In [4]:
# 初始化状态概率矩阵
R = {item: {label: 0 for label in ('Count', 'B', 'M', 'E', 'S')} for item in list(string)}
# 遍历语料库，统计目标字符串中各字符状态在语料库中出现的次数
for line in document:
    if line[0] in R.keys():
        R[line[0]]['Count'] += 1
        R[line[0]][line[-2]] += 1
# 计算各目标字符的状态概率
R = {
    key: {
        label: (R[key][label] / R[key]['Count'])
        for label in LABEL
    }
    for key in R.keys()
}
print(json.dumps(R, ensure_ascii=False, indent=2))
with open('./Characteristics/R.json', 'w') as file:
    json.dump(R, file, ensure_ascii=True, indent=2)

{
  "希": {
    "B": 0.893491124260355,
    "M": 0.08284023668639054,
    "E": 0.013313609467455622,
    "S": 0.010355029585798817
  },
  "腊": {
    "B": 0.05952380952380952,
    "M": 0.15476190476190477,
    "E": 0.7023809523809523,
    "S": 0.08333333333333333
  },
  "的": {
    "B": 0.0011945589765326961,
    "M": 0.0008477515317328812,
    "E": 0.010350275519247814,
    "S": 0.9876074139724866
  },
  "经": {
    "B": 0.7191739973162368,
    "M": 0.13195169226181602,
    "E": 0.11197256597584614,
    "S": 0.03690174444610109
  },
  "济": {
    "B": 0.023208469055374593,
    "M": 0.13368621064060804,
    "E": 0.8407980456026058,
    "S": 0.0023072747014115094
  },
  "结": {
    "B": 0.7705209656925032,
    "M": 0.02744599745870394,
    "E": 0.17458703939008893,
    "S": 0.02744599745870394
  },
  "构": {
    "B": 0.15782434239857335,
    "M": 0.034774855104770394,
    "E": 0.8038341506910388,
    "S": 0.0035666518056174765
  },
  "较": {
    "B": 0.5417675544794189,
    "M": 0.0,
    "E": 0

### 计算每个字转移到下一个字的状态概率

In [5]:
# 初始化转移概率矩阵
P = {
    word: {
        label_A: {
            label_B: 0 for label_B in LABEL
        } for label_A in LABEL
    } for word in string
}
# 遍历语料库
for k in range(1, len(document) - 1):
    if document[k][0] in P.keys():
        # 对于每个字的下一个字，统计其状态出现的次数
        P[document[k][0]][document[k][-2]][document[k + 1][-2]] += 1
# 将次数转换为频率
for word in P.keys():
    for label_A in P[word].keys():
        for label_B in P[word][label_A].keys():
            # 可能某个字不存在某种状态，无法统计其下一个字的状态的概率
            try:
                P[word][label_A][label_B] = P[word][label_A][label_B] / sum(P[word][label_A].values())
            except ZeroDivisionError:
                # 将其统一定义为0即可
                P[word][label_A][label_B] = 0
print(json.dumps(P, ensure_ascii=False, indent=2))
with open('./Characteristics/P.json', 'w') as file:
    json.dump(P, file, ensure_ascii=True, indent=2)

{
  "希": {
    "B": {
      "B": 0.0,
      "M": 0.14072847682119205,
      "E": 0.9998644418112489,
      "S": 0.0
    },
    "M": {
      "B": 0.0,
      "M": 0.32142857142857145,
      "E": 0.943378568086102,
      "S": 0.7597619236568527
    },
    "E": {
      "B": 0.2777777777777778,
      "M": 0.0,
      "E": 0.0,
      "S": 0.9790794979079497
    },
    "S": {
      "B": 0.42857142857142855,
      "M": 0.0,
      "E": 0.0,
      "S": 0.9491525423728814
    }
  },
  "腊": {
    "B": {
      "B": 0.0,
      "M": 0.2,
      "E": 0.9523809523809523,
      "S": 0.0
    },
    "M": {
      "B": 0.0,
      "M": 0.5384615384615384,
      "E": 0.9176470588235295,
      "S": 0.0
    },
    "E": {
      "B": 0.4576271186440678,
      "M": 0.0,
      "E": 0.0,
      "S": 0.9859007832898172
    },
    "S": {
      "B": 0.0,
      "M": 0.0,
      "E": 0.0,
      "S": 1.0
    }
  },
  "的": {
    "B": {
      "B": 0.0,
      "M": 0.05161290322580645,
      "E": 0.9996490150484797,
      "S": 0.

### 计算词组复现概率

#### 向前组合

In [6]:
# 初始化向前词组复现次数
W_prev = {
    string[k]: {state_A: {'All': 0} for state_A in LABEL}
    for k in range(len(string))
}
temp = '^' + string + '^'
# 统计词组复现的次数
for line in range(1, len(document) - 1):
    for word in range(1, len(temp) - 1):
        if document[line][0] == temp[word]:
            W_prev[document[line][0]][document[line][-2]]['All'] += 1
            if document[line - 1][0] == temp[word - 1]:
                W_prev[document[line][0]][document[line][-2]].setdefault(document[line - 1][0], 0)
                W_prev[document[line][0]][document[line][-2]][document[line - 1][0]] += 1
# 转换为概率
for word in range(1, len(temp) - 1):
    for state in W_prev[temp[word]].keys():
        W_prev[temp[word]][state].setdefault(temp[word - 1], 0)
        # 处理分母为0的情况，即语料库不存在这2个字的关联
        try:
            W_prev[temp[word]][state][temp[word - 1]] = (
                W_prev[temp[word]][state][temp[word - 1]]
                / W_prev[temp[word]][state]['All']
            )
        except ZeroDivisionError:
            W_prev[temp[word]][state][temp[word - 1]] = 0
print(json.dumps(W_prev, ensure_ascii=False, indent=2))
with open('./Characteristics/W_prev.json', 'w') as file:
    json.dump(W_prev, file, ensure_ascii=True, indent=2)

{
  "希": {
    "B": {
      "All": 1208,
      "^": 0.3021523178807947
    },
    "M": {
      "All": 112,
      "^": 0.026785714285714284
    },
    "E": {
      "All": 18,
      "^": 0.0
    },
    "S": {
      "All": 14,
      "^": 0.21428571428571427
    }
  },
  "腊": {
    "B": {
      "All": 5,
      "希": 0.0
    },
    "M": {
      "All": 13,
      "希": 0.15384615384615385
    },
    "E": {
      "All": 59,
      "希": 0.9661016949152542
    },
    "S": {
      "All": 7,
      "希": 0.0
    }
  },
  "的": {
    "B": {
      "All": 155,
      "腊": 0.0
    },
    "M": {
      "All": 110,
      "腊": 0.0
    },
    "E": {
      "All": 1343,
      "腊": 0.0
    },
    "S": {
      "All": 128147,
      "腊": 4.682122874511303e-05
    }
  },
  "经": {
    "B": {
      "All": 9647,
      "的": 0.12770809578107184
    },
    "M": {
      "All": 1770,
      "的": 0.0
    },
    "E": {
      "All": 1502,
      "的": 0.0
    },
    "S": {
      "All": 495,
      "的": 0.010101010101010102
    }
  },


#### 向后组合

In [7]:
# 初始化向后词组复现次数
W_subs = {
    string[k]: {state_A: {'All': 0} for state_A in LABEL}
    for k in range(len(string))
}
temp = '^' + string + '^'
# 统计词组复现的次数
for line in range(1, len(document) - 1):
    for word in range(1, len(temp) - 1):
        if document[line][0] == temp[word]:
            W_subs[document[line][0]][document[line][-2]]['All'] += 1
            if document[line + 1][0] == temp[word + 1]:
                W_subs[document[line][0]][
                    document[line][-2]
                ].setdefault(document[line + 1][0], 0)
                W_subs[document[line][0]][
                    document[line][-2]
                ][document[line + 1][0]] += 1
# 转换为概率
for word in range(1, len(temp) - 1):
    for state in W_subs[temp[word]].keys():
        W_subs[temp[word]][state].setdefault(temp[word + 1], 0)
        # 处理分母为0的情况，即语料库不存在这2个字的关联
        try:
            W_subs[temp[word]][state][temp[word + 1]] = (
                W_subs[temp[word]][state][temp[word + 1]]
                / W_subs[temp[word]][state]['All']
            )
        except ZeroDivisionError:
            W_subs[temp[word]][state][temp[word + 1]] = 0
print(json.dumps(W_subs, ensure_ascii=False, indent=2))
# with open('./Characteristics/W_subs.json', 'w') as file:
#     json.dump(W_subs, file, ensure_ascii=True, indent=2)

{
  "希": {
    "B": {
      "All": 1208,
      "腊": 0.048841059602649006
    },
    "M": {
      "All": 112,
      "腊": 0.0
    },
    "E": {
      "All": 18,
      "腊": 0.0
    },
    "S": {
      "All": 14,
      "腊": 0.0
    }
  },
  "腊": {
    "B": {
      "All": 5,
      "的": 0.0
    },
    "M": {
      "All": 13,
      "的": 0.0
    },
    "E": {
      "All": 59,
      "的": 0.1016949152542373
    },
    "S": {
      "All": 7,
      "的": 0.0
    }
  },
  "的": {
    "B": {
      "All": 155,
      "经": 0.0
    },
    "M": {
      "All": 110,
      "经": 0.0
    },
    "E": {
      "All": 1343,
      "经": 0.0022338049143708115
    },
    "S": {
      "All": 128147,
      "经": 0.009629566045244915
    }
  },
  "经": {
    "B": {
      "All": 9647,
      "济": 0.596143878926091
    },
    "M": {
      "All": 1770,
      "济": 0.7214689265536723
    },
    "E": {
      "All": 1502,
      "济": 0.0
    },
    "S": {
      "All": 495,
      "济": 0.0
    }
  },
  "济": {
    "B": {
      "All": 1

## 开始分词

### 输入语句并切分为语素

$4070472$行数据对于`Python3`来说实在过于庞大，如果对语料库整体进行处理，每一步产生的数据量都是爆炸式的。所以，我们将目标语句的输入放到特征训练之前，对目标语句执行针对性训练。这样可以剔除分词时不会被使用的数据，极大程度地压缩训练过程中产生的数据量。

### 初始化字与状态的初始矩阵映射关系表

In [8]:
S = [{state: {'Rate': 0, 'Path': 0} for state in LABEL} for word in string]

### 计算字与状态对应关系

In [9]:
# 创建临时字符串，防止越界
temp = '^' + string + '^'
# 首字不存在其它字转移到它的概率，需要单独处理
for state in LABEL:
    S[0][state]['Rate'] = (
        W_prev[string[0]][state]['^']
        + W_subs[string[0]][state][string[1]]
        + R[string[0]][state]
    )
for word in range(2, len(temp) - 1):
    for state_A in LABEL:
        routes = [
            P[temp[word - 1]][state_B][state_A] * S[word - 2][state_B]['Rate']
            for state_B in LABEL
        ]
        maximum = max(routes)
        S[word - 1][state_A]['Path'] = LABEL[routes.index(maximum)]
        S[word - 1][state_A]['Rate'] = (
            maximum
            + W_prev[temp[word]][state_A][temp[word - 1]]
            + W_subs[temp[word]][state_A][temp[word + 1]]
            + R[temp[word]][state_A]
        )
print(json.dumps(S, ensure_ascii=False, indent=2))
with open('./Characteristics/S.json', 'w') as file:
    json.dump(S, file, ensure_ascii=True, indent=2)

[
  {
    "B": {
      "Rate": 1.2444845017437989,
      "Path": 0
    },
    "M": {
      "Rate": 0.10962595097210483,
      "Path": 0
    },
    "E": {
      "Rate": 0.013313609467455622,
      "Path": 0
    },
    "S": {
      "Rate": 0.2246407438715131,
      "Path": 0
    }
  },
  {
    "B": {
      "Rate": 0.15579841404017225,
      "Path": "S"
    },
    "M": {
      "Rate": 0.48374246696604356,
      "Path": "B"
    },
    "E": {
      "Rate": 3.0144933642292573,
      "Path": "B"
    },
    "S": {
      "Rate": 0.29655166649951525,
      "Path": "S"
    }
  },
  {
    "B": {
      "Rate": 1.38070847142043,
      "Path": "E"
    },
    "M": {
      "Rate": 0.26132446451344865,
      "Path": "M"
    },
    "E": {
      "Rate": 0.4564889324730469,
      "Path": "M"
    },
    "S": {
      "Rate": 3.969275170262058,
      "Path": "E"
    }
  },
  {
    "B": {
      "Rate": 4.77847219733105,
      "Path": "S"
    },
    "M": {
      "Rate": 0.9579504046208678,
      "Path": "M"
   

### 获得分词结果

In [10]:
result = []
temp = [rate['Rate'] for rate in S[-1].values()]
index = temp.index(max(temp))
current_state = tuple(S[-1].keys())[index]
result.append(current_state)
for word in range(len(string) - 1, -1, -1):
    result.append(string[word])
    result.append(S[word][current_state]['Path'])
result.pop()
result.reverse()
result = ''.join(result)
print(result)
result.replace('B', '').replace('M', '').replace('E', ' ').replace('S', ' ')

希S腊E的S经S济E结S构E较S特S殊S


'希 腊 的 经 济 结 构 较 特 殊 '